# 🧠 Memória Cívica - Teste de Viabilidade Técnica

Este notebook valida se os dados abertos da Câmara dos Deputados são suficientes para construir o MVP do projeto.

**Objetivo:** Conseguir montar uma tabela com:
- Nome da proposição
- Data da votação
- Placar (SIM/NÃO)
- Lista de deputados com seus votos

Se conseguirmos isso, o projeto é **100% viável**.

## 1. Setup e Imports

In [18]:
# Instalar dependências (execute apenas uma vez)
# !pip install pandas requests tqdm

In [19]:
import os

import pandas as pd
import requests

# Configurações de exibição
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', None)

# Diretório para cache dos dados
DATA_DIR = './dados_camara'
os.makedirs(DATA_DIR, exist_ok=True)

print("✅ Setup completo!")

✅ Setup completo!


## 2. Download dos Dados

Vamos baixar os arquivos CSV de 2024 (ano completo) para ter dados suficientes para análise.

**Arquivos necessários:**
- `votacoes-2024.csv` - Metadados das votações
- `votacoesVotos-2024.csv` - Votos individuais de cada deputado
- `votacoesProposicoes-2024.csv` - Link entre votações e proposições
- `proposicoes-2024.csv` - Detalhes das proposições (PLs, etc.)
- `deputados.csv` - Dados dos parlamentares

In [20]:
# URLs base dos arquivos
BASE_URL = "https://dadosabertos.camara.leg.br/arquivos"

ARQUIVOS = {
    'votacoes': f"{BASE_URL}/votacoes/csv/votacoes-2024.csv",
    'votos': f"{BASE_URL}/votacoesVotos/csv/votacoesVotos-2024.csv",
    'votacoes_proposicoes': f"{BASE_URL}/votacoesProposicoes/csv/votacoesProposicoes-2024.csv",
    'proposicoes': f"{BASE_URL}/proposicoes/csv/proposicoes-2024.csv",
    'deputados': f"{BASE_URL}/deputados/csv/deputados.csv",
    'orientacoes': f"{BASE_URL}/votacoesOrientacoes/csv/votacoesOrientacoes-2024.csv",
}

def baixar_arquivo(nome, url):
    """Baixa arquivo se não existir no cache local."""
    filepath = os.path.join(DATA_DIR, f"{nome}.csv")

    if os.path.exists(filepath):
        print(f"📁 {nome}: usando cache local")
        return filepath

    print(f"⬇️  {nome}: baixando...")
    try:
        response = requests.get(url, timeout=60)
        response.raise_for_status()

        with open(filepath, 'wb') as f:
            f.write(response.content)

        size_mb = len(response.content) / (1024 * 1024)
        print(f"   ✅ {size_mb:.2f} MB baixados")
        return filepath
    except Exception as e:
        print(f"   ❌ Erro: {e}")
        return None

# Baixar todos os arquivos
print("=" * 50)
print("BAIXANDO DADOS DA CÂMARA DOS DEPUTADOS (2024)")
print("=" * 50)

arquivos_locais = {}
for nome, url in ARQUIVOS.items():
    arquivos_locais[nome] = baixar_arquivo(nome, url)

print("\n✅ Download concluído!")

BAIXANDO DADOS DA CÂMARA DOS DEPUTADOS (2024)
📁 votacoes: usando cache local
📁 votos: usando cache local
📁 votacoes_proposicoes: usando cache local
📁 proposicoes: usando cache local
📁 deputados: usando cache local
📁 orientacoes: usando cache local

✅ Download concluído!


## 3. Carregar e Explorar os Dados

In [21]:
# Carregar DataFrames
print("Carregando dados...\n")

# Usar caminhos diretos para evitar problemas com downloads falhados
df_votacoes = pd.read_csv(os.path.join(DATA_DIR, 'votacoes.csv'), sep=';', encoding='utf-8')
df_votos = pd.read_csv(os.path.join(DATA_DIR, 'votos.csv'), sep=';', encoding='utf-8')
df_votacoes_proposicoes = pd.read_csv(os.path.join(DATA_DIR, 'votacoes_proposicoes.csv'), sep=';', encoding='utf-8')
df_proposicoes = pd.read_csv(os.path.join(DATA_DIR, 'proposicoes.csv'), sep=';', encoding='utf-8')
df_deputados = pd.read_csv(os.path.join(DATA_DIR, 'deputados.csv'), sep=';', encoding='utf-8')
df_orientacoes = pd.read_csv(os.path.join(DATA_DIR, 'orientacoes.csv'), sep=';', encoding='utf-8')

print("📊 RESUMO DOS DADOS CARREGADOS:")
print(f"   • Votações: {len(df_votacoes):,} registros")
print(f"   • Votos individuais: {len(df_votos):,} registros")
print(f"   • Votações-Proposições: {len(df_votacoes_proposicoes):,} registros")
print(f"   • Proposições: {len(df_proposicoes):,} registros")
print(f"   • Deputados: {len(df_deputados):,} registros")
print(f"   • Orientações de bancada: {len(df_orientacoes):,} registros")

Carregando dados...

📊 RESUMO DOS DADOS CARREGADOS:
   • Votações: 10,371 registros
   • Votos individuais: 116,400 registros
   • Votações-Proposições: 7,430 registros
   • Proposições: 61,508 registros
   • Deputados: 7,876 registros
   • Orientações de bancada: 4,361 registros


/var/folders/27/bdy8dsxs5z569cls38mw75180000gs/T/ipykernel_67060/1785008411.py:8: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_proposicoes = pd.read_csv(os.path.join(DATA_DIR, 'proposicoes.csv'), sep=';', encoding='utf-8')


### 3.1 Estrutura: Votações

In [22]:
print("🗳️ ESTRUTURA - VOTAÇÕES")
print("Colunas disponíveis:")
print(df_votacoes.columns.tolist())
print("\nExemplo de registro:")
df_votacoes.head(2)

🗳️ ESTRUTURA - VOTAÇÕES
Colunas disponíveis:
['id', 'uri', 'data', 'dataHoraRegistro', 'idOrgao', 'uriOrgao', 'siglaOrgao', 'idEvento', 'uriEvento', 'aprovacao', 'votosSim', 'votosNao', 'votosOutros', 'descricao', 'ultimaAberturaVotacao_dataHoraRegistro', 'ultimaAberturaVotacao_descricao', 'ultimaApresentacaoProposicao_dataHoraRegistro', 'ultimaApresentacaoProposicao_descricao', 'ultimaApresentacaoProposicao_idProposicao', 'ultimaApresentacaoProposicao_uriProposicao']

Exemplo de registro:


,id,uri,data,dataHoraRegistro,idOrgao,uriOrgao,siglaOrgao,idEvento,uriEvento,aprovacao,votosSim,votosNao,votosOutros,descricao,ultimaAberturaVotacao_dataHoraRegistro,ultimaAberturaVotacao_descricao,ultimaApresentacaoProposicao_dataHoraRegistro,ultimaApresentacaoProposicao_descricao,ultimaApresentacaoProposicao_idProposicao,ultimaApresentacaoProposicao_uriProposicao
0,2355879-47,https://dadosabertos.camara.leg.br/api/v2/votacoes/2355879-47,2024-02-06,2024-02-06T18:35:59,100001,https://dadosabertos.camara.leg.br/api/v2/orgaos/100001,SECAP(SGM),0,NaN,1.0,0,0,0,"Realizar o encaminhamento do PL-1825/2023 à CCJC (tramitação simultânea), em razão da aprovação ...",NaN,NaN,NaN,NaN,0,NaN
1,2355879-48,https://dadosabertos.camara.leg.br/api/v2/votacoes/2355879-48,2024-02-06,2024-02-06T18:36:00,100001,https://dadosabertos.camara.leg.br/api/v2/orgaos/100001,SECAP(SGM),0,NaN,1.0,0,0,0,"Realizar o encaminhamento do PL-1825/2023 à CE (tramitação simultânea), em razão da aprovação de...",NaN,NaN,NaN,NaN,0,NaN


### 3.2 Estrutura: Votos Individuais

In [23]:
print("🧑 ESTRUTURA - VOTOS INDIVIDUAIS")
print("Colunas disponíveis:")
print(df_votos.columns.tolist())
print("\nExemplo de registro:")
df_votos.head(2)

🧑 ESTRUTURA - VOTOS INDIVIDUAIS
Colunas disponíveis:
['idVotacao', 'uriVotacao', 'dataHoraVoto', 'voto', 'deputado_id', 'deputado_uri', 'deputado_nome', 'deputado_siglaPartido', 'deputado_uriPartido', 'deputado_siglaUf', 'deputado_idLegislatura', 'deputado_urlFoto']

Exemplo de registro:


,idVotacao,uriVotacao,dataHoraVoto,voto,deputado_id,deputado_uri,deputado_nome,deputado_siglaPartido,deputado_uriPartido,deputado_siglaUf,deputado_idLegislatura,deputado_urlFoto
0,1197773-140,https://dadosabertos.camara.leg.br/api/v2/votacoes/1197773-140,2024-12-04T21:10:04,Não,204379,https://dadosabertos.camara.leg.br/api/v2/deputados/204379,Acácio Favacho,MDB,https://dadosabertos.camara.leg.br/api/v2/partidos/36899,AP,57,https://www.camara.leg.br/internet/deputado/bandep/204379.jpg
1,1197773-140,https://dadosabertos.camara.leg.br/api/v2/votacoes/1197773-140,2024-12-04T21:08:25,Não,220714,https://dadosabertos.camara.leg.br/api/v2/deputados/220714,Adail Filho,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/partidos/37908,AM,57,https://www.camara.leg.br/internet/deputado/bandep/220714.jpg


### 3.3 Estrutura: Proposições

In [26]:
print("📜 ESTRUTURA - PROPOSIÇÕES")
print("Colunas disponíveis:")
print(df_proposicoes.columns.tolist())
print("\nExemplo de registro:")
# Ajuste: o arquivo votacoesProposicoes tem os dados de proposições com prefixo 'proposicao_'
colunas_prop = ['proposicao_id', 'proposicao_siglaTipo', 'proposicao_numero', 'proposicao_ano', 'proposicao_ementa']
colunas_existentes = [c for c in colunas_prop if c in df_proposicoes.columns]
if colunas_existentes:
    df_proposicoes[colunas_existentes].head(2)
else:
    # Fallback para nomes antigos
    df_proposicoes[['id', 'siglaTipo', 'numero', 'ano', 'ementa']].head(2)

📜 ESTRUTURA - PROPOSIÇÕES
Colunas disponíveis:
['id', 'uri', 'siglaTipo', 'numero', 'ano', 'codTipo', 'descricaoTipo', 'ementa', 'ementaDetalhada', 'keywords', 'dataApresentacao', 'uriOrgaoNumerador', 'uriPropAnterior', 'uriPropPrincipal', 'uriPropPosterior', 'urlInteiroTeor', 'urnFinal', 'ultimoStatus_dataHora', 'ultimoStatus_sequencia', 'ultimoStatus_uriRelator', 'ultimoStatus_idOrgao', 'ultimoStatus_siglaOrgao', 'ultimoStatus_uriOrgao', 'ultimoStatus_regime', 'ultimoStatus_descricaoTramitacao', 'ultimoStatus_idTipoTramitacao', 'ultimoStatus_descricaoSituacao', 'ultimoStatus_idSituacao', 'ultimoStatus_despacho', 'ultimoStatus_apreciacao', 'ultimoStatus_url']

Exemplo de registro:


### 3.4 Tipos de Voto Disponíveis

In [27]:
print("📊 TIPOS DE VOTO REGISTRADOS:")
print(df_votos['voto'].value_counts())

📊 TIPOS DE VOTO REGISTRADOS:
voto
Sim          67580
Não          46630
Obstrução     1640
Abstenção      294
Artigo 17      256
Name: count, dtype: int64


## 4. Teste de Viabilidade: Montar o "Watch Item"

Agora vamos fazer o JOIN dos dados para criar a estrutura que você precisa para o MVP.

In [32]:
# Identificar a coluna de ID da votação em cada DataFrame
print("🔍 Identificando colunas de ID para JOIN:")
print(f"   df_votacoes: {[c for c in df_votacoes.columns if 'id' in c.lower()]}")
print(f"   df_votos: {[c for c in df_votos.columns if 'id' in c.lower()]}")
print(f"   df_votacoes_proposicoes: {[c for c in df_votacoes_proposicoes.columns if 'id' in c.lower()]}")
print(f"   df_proposicoes: {[c for c in df_proposicoes.columns if 'id' in c.lower()]}")

🔍 Identificando colunas de ID para JOIN:
   df_votacoes: ['id', 'idOrgao', 'idEvento', 'ultimaApresentacaoProposicao_idProposicao']
   df_votos: ['idVotacao', 'deputado_id', 'deputado_siglaPartido', 'deputado_uriPartido', 'deputado_idLegislatura']
   df_votacoes_proposicoes: ['idVotacao', 'proposicao_id']
   df_proposicoes: ['id', 'ultimoStatus_idOrgao', 'ultimoStatus_idTipoTramitacao', 'ultimoStatus_idSituacao']


In [33]:
# Função para extrair ID da URI (se necessário)
def extrair_id_uri(uri):
    """Extrai o ID numérico de uma URI da API."""
    if pd.isna(uri):
        return None
    return str(uri).split('/')[-1]

# Verificar formato das colunas de ID
print("Amostra de IDs:")
if 'idVotacao' in df_votacoes.columns:
    print(f"   votacoes.idVotacao: {df_votacoes['idVotacao'].iloc[0]}")
if 'idVotacao' in df_votos.columns:
    print(f"   votos.idVotacao: {df_votos['idVotacao'].iloc[0]}")
if 'uriVotacao' in df_votacoes_proposicoes.columns:
    print(f"   votacoes_proposicoes.uriVotacao: {df_votacoes_proposicoes['uriVotacao'].iloc[0]}")

Amostra de IDs:
   votos.idVotacao: 1197773-140
   votacoes_proposicoes.uriVotacao: https://dadosabertos.camara.leg.br/api/v2/votacoes/1006391-32


In [34]:
# Filtrar apenas votações NOMINAIS do PLENÁRIO (escopo do MVP)
print("🎯 Filtrando votações nominais do Plenário...")

# Verificar colunas disponíveis para filtro
print(f"\nColunas de votacoes: {df_votacoes.columns.tolist()}")

# Filtrar votações nominais (onde há votos individuais registrados)
votacoes_com_votos = df_votos['idVotacao'].unique()
print(f"\nVotações com votos registrados: {len(votacoes_com_votos)}")

# Usar coluna 'id' do arquivo votacoes.csv
col_id_votacao = 'id' if 'id' in df_votacoes.columns else 'idVotacao'
df_votacoes_nominais = df_votacoes[df_votacoes[col_id_votacao].isin(votacoes_com_votos)].copy()
# Renomear para padronizar
if col_id_votacao == 'id':
    df_votacoes_nominais = df_votacoes_nominais.rename(columns={'id': 'idVotacao'})
print(f"Votações nominais encontradas: {len(df_votacoes_nominais)}")

🎯 Filtrando votações nominais do Plenário...

Colunas de votacoes: ['id', 'uri', 'data', 'dataHoraRegistro', 'idOrgao', 'uriOrgao', 'siglaOrgao', 'idEvento', 'uriEvento', 'aprovacao', 'votosSim', 'votosNao', 'votosOutros', 'descricao', 'ultimaAberturaVotacao_dataHoraRegistro', 'ultimaAberturaVotacao_descricao', 'ultimaApresentacaoProposicao_dataHoraRegistro', 'ultimaApresentacaoProposicao_descricao', 'ultimaApresentacaoProposicao_idProposicao', 'ultimaApresentacaoProposicao_uriProposicao']

Votações com votos registrados: 448
Votações nominais encontradas: 448


In [35]:
# Calcular placar de cada votação
print("📊 Calculando placar das votações...")

placar = df_votos.groupby('idVotacao')['voto'].value_counts().unstack(fill_value=0)
placar = placar.reset_index()

print(f"\nColunas do placar: {placar.columns.tolist()}")
placar.head()

📊 Calculando placar das votações...

Colunas do placar: ['idVotacao', 'Abstenção', 'Artigo 17', 'Não', 'Obstrução', 'Sim']


voto,idVotacao,Abstenção,Artigo 17,Não,Obstrução,Sim
0,1197773-140,1,1,404,0,23
1,1212491-84,0,0,20,0,31
2,1301141-69,0,0,38,0,3
3,1548579-186,0,1,332,0,36
4,1548579-194,2,1,38,0,336


In [36]:
# Juntar votações com placar
df_votacoes_placar = df_votacoes_nominais.merge(
    placar,
    on='idVotacao',
    how='left'
)

print(f"✅ Votações com placar: {len(df_votacoes_placar)}")
df_votacoes_placar.head(2)

✅ Votações com placar: 448


,idVotacao,uri,data,dataHoraRegistro,idOrgao,uriOrgao,siglaOrgao,idEvento,uriEvento,aprovacao,votosSim,votosNao,votosOutros,descricao,ultimaAberturaVotacao_dataHoraRegistro,ultimaAberturaVotacao_descricao,ultimaApresentacaoProposicao_dataHoraRegistro,ultimaApresentacaoProposicao_descricao,ultimaApresentacaoProposicao_idProposicao,ultimaApresentacaoProposicao_uriProposicao,Abstenção,Artigo 17,Não,Obstrução,Sim
0,2367548-7,https://dadosabertos.camara.leg.br/api/v2/votacoes/2367548-7,2024-02-20,2024-02-20T21:12:58,180,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,PLEN,71846,https://dadosabertos.camara.leg.br/api/v2/eventos/71846,1.0,442,2,1,Aprovado o Requerimento de Urgência (Art. 155 do RICD). Sim: 442; não: 2; total: 444.,NaN,NaN,NaN,NaN,0,NaN,0,1,2,0,442
1,2388837-35,https://dadosabertos.camara.leg.br/api/v2/votacoes/2388837-35,2024-02-21,2024-02-21T19:43:55,180,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,PLEN,71852,https://dadosabertos.camara.leg.br/api/v2/eventos/71852,1.0,425,0,1,"Aprovado, em apreciação preliminar, o Parecer da Comissão Mista de Planos, Orçamentos Públicos e...",2024-02-21T19:33:55,Votação preliminar em turno único.,NaN,NaN,0,NaN,0,1,0,0,425


In [37]:
# Preparar link votação -> proposição
print("🔗 Verificando link votação → proposição...")

# O arquivo votacoes_proposicoes já tem idVotacao e as colunas de proposição com prefixo
print(f"Colunas disponíveis: {df_votacoes_proposicoes.columns.tolist()}")
print(f"\nLinks votação-proposição: {len(df_votacoes_proposicoes)}")
df_votacoes_proposicoes[['idVotacao', 'proposicao_id', 'proposicao_siglaTipo']].head()

🔗 Verificando link votação → proposição...
Colunas disponíveis: ['idVotacao', 'uriVotacao', 'data', 'descricao', 'proposicao_id', 'proposicao_uri', 'proposicao_titulo', 'proposicao_ementa', 'proposicao_codTipo', 'proposicao_siglaTipo', 'proposicao_numero', 'proposicao_ano']

Links votação-proposição: 7430


,idVotacao,proposicao_id,proposicao_siglaTipo
0,1006391-32,1006391,PL
1,1029052-77,1029052,PL
2,1030398-71,1030398,PL
3,1030398-74,1030398,PL
4,1050899-75,1050899,PL


In [38]:
# Converter IDs para mesmo tipo
df_votacoes_placar['idVotacao'] = df_votacoes_placar['idVotacao'].astype(str)
df_votacoes_proposicoes['idVotacao'] = df_votacoes_proposicoes['idVotacao'].astype(str)

# Juntar votações com proposições (usar idVotacao que já existe)
df_completo = df_votacoes_placar.merge(
    df_votacoes_proposicoes[['idVotacao', 'proposicao_id', 'proposicao_siglaTipo', 'proposicao_numero', 'proposicao_ano', 'proposicao_ementa']],
    on='idVotacao',
    how='left'
)

print(f"Após merge com proposições: {len(df_completo)}")

Após merge com proposições: 485


In [39]:
# Renomear colunas para simplificar
df_completo = df_completo.rename(columns={
    'proposicao_id': 'id_proposicao',
    'proposicao_siglaTipo': 'siglaTipo',
    'proposicao_numero': 'numero',
    'proposicao_ano': 'ano',
    'proposicao_ementa': 'ementa'
})

print(f"\n✅ Dataset completo: {len(df_completo)} registros")
print(f"\nColunas finais: {df_completo.columns.tolist()}")


✅ Dataset completo: 485 registros

Colunas finais: ['idVotacao', 'uri', 'data', 'dataHoraRegistro', 'idOrgao', 'uriOrgao', 'siglaOrgao', 'idEvento', 'uriEvento', 'aprovacao', 'votosSim', 'votosNao', 'votosOutros', 'descricao', 'ultimaAberturaVotacao_dataHoraRegistro', 'ultimaAberturaVotacao_descricao', 'ultimaApresentacaoProposicao_dataHoraRegistro', 'ultimaApresentacaoProposicao_descricao', 'ultimaApresentacaoProposicao_idProposicao', 'ultimaApresentacaoProposicao_uriProposicao', 'Abstenção', 'Artigo 17', 'Não', 'Obstrução', 'Sim', 'id_proposicao', 'siglaTipo', 'numero', 'ano', 'ementa']


## 5. Resultado: Watch Item Completo

Agora vamos visualizar alguns exemplos do "Watch Item" que seria exibido no feed do produto.

In [40]:
# Selecionar colunas relevantes para o Watch Item
colunas_watch_item = [
    'idVotacao',
    'dataHoraRegistro',
    'descricao',
    'siglaTipo',
    'numero',
    'ano',
    'ementa',
]

# Adicionar colunas de placar se existirem
for col in ['Sim', 'Não', 'Abstenção', 'Obstrução']:
    if col in df_completo.columns:
        colunas_watch_item.append(col)

# Filtrar apenas registros com proposição identificada
df_watch = df_completo[df_completo['ementa'].notna()][colunas_watch_item].drop_duplicates()

print(f"📋 WATCH ITEMS PRONTOS: {len(df_watch)}")
print("\n" + "="*80)
df_watch.head(10)

📋 WATCH ITEMS PRONTOS: 484



,idVotacao,dataHoraRegistro,descricao,siglaTipo,numero,ano,ementa,Sim,Não,Abstenção,Obstrução
0,2367548-7,2024-02-20T21:12:58,Aprovado o Requerimento de Urgência (Art. 155 do RICD). Sim: 442; não: 2; total: 444.,PL,10106.0,2018.0,"Altera a Lei nº 8.080, de 19 de setembro de 1990, para dispor sobre a obrigatoriedade de publica...",442,2,0,0
1,2367548-7,2024-02-20T21:12:58,Aprovado o Requerimento de Urgência (Art. 155 do RICD). Sim: 442; não: 2; total: 444.,REQ,1884.0,2023.0,"Requer urgência para o PL 10106/2018, do Senado Federal, que “altera a Lei nº 8.080, de 19 de se...",442,2,0,0
2,2388837-35,2024-02-21T19:43:55,"Aprovado, em apreciação preliminar, o Parecer da Comissão Mista de Planos, Orçamentos Públicos e...",MPV,1188.0,2023.0,"Abre crédito extraordinário, em favor dos Ministérios da Defesa, da Integração e do Desenvolvime...",425,0,0,0
3,2382257-49,2024-02-21T20:50:49,"Aprovada a Subemenda Substitutiva Global ao Projeto de Lei Complementar nº 175, de 2023, adotada...",PLP,175.0,2023.0,"Altera o artigo 5º da Lei Complementar nº 172, de 15 de abril de 2020, para conceder prazo para ...",426,4,0,0
4,2411815-7,2024-02-21T21:02:11,Aprovado o Requerimento de Urgência (Art. 155 do RICD). Sim: 413; não: 13; abstenção: 1; total: ...,PL,1269.0,2022.0,"Acrescenta o Art. 16-A na Lei Federal nº 8.429 de 02 de junho de 1992, que dispõe sobre as sançõ...",413,13,1,0
5,2403068-11,2024-02-27T19:00:29,Aprovado o Requerimento de Urgência (Art. 155 do RICD). Sim: 285; não: 103; abstenção: 1; total:...,PL,4932.0,2023.0,Estabelece requisitos para a autorização do funcionamento de prestadores de serviços de ativos v...,285,103,1,0
6,2403068-11,2024-02-27T19:00:29,Aprovado o Requerimento de Urgência (Art. 155 do RICD). Sim: 285; não: 103; abstenção: 1; total:...,REQ,3912.0,2023.0,"Requer urgência do art. 155 para apreciação imediata do PL 4.932, de 2023.",285,103,1,0
7,2379378-9,2024-02-27T20:44:54,Aprovado o Requerimento de Urgência (Art. 155 do RICD). Sim: 258; não: 164; abstenção: 3; total:...,PL,2459.0,2023.0,Estabelece a obrigatoriedade de afixação de cartazes e veiculação de mensagem sonoro nos portos ...,258,164,3,0
8,2415411-9,2024-02-27T21:09:19,Aprovado o Requerimento de Urgência (Art. 155 do RICD). Sim: 339; não: 82; abstenção: 1; total: ...,PL,4831.0,2023.0,"Altera a Lei no 9.074, de 7 de julho de 1995 e a Lei no 10.848, de 15 de março de 2004, e dá out...",339,82,1,0
9,2415411-9,2024-02-27T21:09:19,Aprovado o Requerimento de Urgência (Art. 155 do RICD). Sim: 339; não: 82; abstenção: 1; total: ...,REQ,4403.0,2023.0,"Requer regime de urgência para apreciação do PL nº 4831 de 2023, que “Altera a Lei nº 9.074, de ...",339,82,1,0


In [41]:
# Exibir um Watch Item formatado
def formatar_watch_item(row):
    """Formata um registro como Watch Item para visualização."""
    print("=" * 80)
    print(f"📜 {row.get('siglaTipo', 'N/A')} {row.get('numero', 'N/A')}/{row.get('ano', 'N/A')}")
    print(f"📅 Data: {row.get('dataHoraRegistro', 'N/A')}")
    print(f"\n📝 Ementa: {row.get('ementa', 'N/A')[:200]}...")
    print(f"\n🗳️ Descrição da votação: {row.get('descricao', 'N/A')[:150]}...")

    sim = row.get('Sim', 0)
    nao = row.get('Não', 0)
    print(f"\n📊 PLACAR: ✅ SIM: {sim}  |  ❌ NÃO: {nao}")
    print("=" * 80)

# Mostrar 3 exemplos
print("\n🔍 EXEMPLOS DE WATCH ITEMS:\n")
for idx, row in df_watch.head(3).iterrows():
    formatar_watch_item(row)
    print()


🔍 EXEMPLOS DE WATCH ITEMS:

📜 PL 10106.0/2018.0
📅 Data: 2024-02-20T21:12:58

📝 Ementa: Altera a Lei nº 8.080, de 19 de setembro de 1990, para dispor sobre a obrigatoriedade de publicação na internet de listas de pacientes que serão submetidos a procedimentos cirúrgicos eletivos no âmbit...

🗳️ Descrição da votação: Aprovado o Requerimento de Urgência (Art. 155 do RICD). Sim: 442; não: 2; total: 444....

📊 PLACAR: ✅ SIM: 442  |  ❌ NÃO: 2

📜 REQ 1884.0/2023.0
📅 Data: 2024-02-20T21:12:58

📝 Ementa: Requer urgência para o PL 10106/2018, do Senado Federal, que “altera a Lei nº 8.080, de 19 de setembro de 1990, para dispor sobre a obrigatoriedade de publicação na internet de listas de pacientes que...

🗳️ Descrição da votação: Aprovado o Requerimento de Urgência (Art. 155 do RICD). Sim: 442; não: 2; total: 444....

📊 PLACAR: ✅ SIM: 442  |  ❌ NÃO: 2

📜 MPV 1188.0/2023.0
📅 Data: 2024-02-21T19:43:55

📝 Ementa: Abre crédito extraordinário, em favor dos Ministérios da Defesa, da Integração e do 

## 6. Teste: Votos Individuais de uma Votação

Agora vamos pegar uma votação específica e listar como cada deputado votou.

In [42]:
# Pegar uma votação de exemplo
id_votacao_exemplo = df_watch['idVotacao'].iloc[0]
print(f"🔍 Analisando votação: {id_votacao_exemplo}")

# Filtrar votos dessa votação
df_votos['idVotacao'] = df_votos['idVotacao'].astype(str)
votos_exemplo = df_votos[df_votos['idVotacao'] == str(id_votacao_exemplo)].copy()

print(f"\nTotal de votos registrados: {len(votos_exemplo)}")
print(f"\nColunas disponíveis nos votos: {votos_exemplo.columns.tolist()}")

🔍 Analisando votação: 2367548-7

Total de votos registrados: 445

Colunas disponíveis nos votos: ['idVotacao', 'uriVotacao', 'dataHoraVoto', 'voto', 'deputado_id', 'deputado_uri', 'deputado_nome', 'deputado_siglaPartido', 'deputado_uriPartido', 'deputado_siglaUf', 'deputado_idLegislatura', 'deputado_urlFoto']


In [43]:
# Exibir votos por partido
print("📊 VOTOS POR PARTIDO:")
# Usar nome correto da coluna (deputado_siglaPartido no arquivo votacoesVotos)
col_partido = 'deputado_siglaPartido' if 'deputado_siglaPartido' in votos_exemplo.columns else 'siglaPartido'
print(votos_exemplo.groupby([col_partido, 'voto']).size().unstack(fill_value=0))

📊 VOTOS POR PARTIDO:
voto                   Artigo 17  Não  Sim
deputado_siglaPartido                     
AVANTE                         0    0    6
CIDADANIA                      0    0    4
MDB                            0    0   33
NOVO                           0    0    3
PCdoB                          0    0    7
PDT                            0    0   18
PL                             0    0   88
PODE                           0    0   13
PP                             1    1   42
PRD                            0    0    4
PSB                            0    0   13
PSD                            0    0   38
PSDB                           0    0   12
PSOL                           0    0   12
PT                             0    0   60
PV                             0    0    5
REDE                           0    0    1
REPUBLICANOS                   0    0   35
SOLIDARIEDADE                  0    1    4
UNIÃO                          0    0   44


In [44]:
# Exibir votos individuais (amostra)
print("\n👤 VOTOS INDIVIDUAIS (primeiros 20):")
# Nomes das colunas no arquivo votacoesVotos têm prefixo 'deputado_'
colunas_voto = ['deputado_nome', 'deputado_siglaPartido', 'deputado_siglaUf', 'voto']
colunas_disponiveis = [c for c in colunas_voto if c in votos_exemplo.columns]

votos_exemplo[colunas_disponiveis].head(20)


👤 VOTOS INDIVIDUAIS (primeiros 20):


,deputado_nome,deputado_siglaPartido,deputado_siglaUf,voto
32712,Abilio Brunini,PL,MT,Sim
32713,Acácio Favacho,MDB,AP,Sim
32714,Adail Filho,REPUBLICANOS,AM,Sim
32715,Adilson Barroso,PL,SP,Sim
32716,Adriana Ventura,NOVO,SP,Sim
32717,Adriano do Baldy,PP,GO,Sim
32718,Afonso Hamm,PP,RS,Sim
32719,Afonso Motta,PDT,RS,Sim
32720,AJ Albuquerque,PP,CE,Sim
32721,Alberto Fraga,PL,DF,Sim


## 7. Teste: Orientação de Bancada

Verificar se temos as orientações dos líderes partidários.

In [45]:
print("🏛️ ORIENTAÇÕES DE BANCADA")
print(f"\nColunas disponíveis: {df_orientacoes.columns.tolist()}")
print(f"\nTotal de orientações: {len(df_orientacoes)}")

df_orientacoes.head(10)

🏛️ ORIENTAÇÕES DE BANCADA

Colunas disponíveis: ['idVotacao', 'uriVotacao', 'siglaOrgao', 'descricao', 'siglaBancada', 'uriBancada', 'orientacao']

Total de orientações: 4361


,idVotacao,uriVotacao,siglaOrgao,descricao,siglaBancada,uriBancada,orientacao
0,2460010-43,https://dadosabertos.camara.leg.br/api/v2/votacoes/2460010-43,PLEN,"Aprovado o Projeto de Lei nº 3.802, de 2024. Sim: 340; Não: 117; Abstenção: 1; Total: 458.",Minoria,NaN,Não
1,2460010-43,https://dadosabertos.camara.leg.br/api/v2/votacoes/2460010-43,PLEN,"Aprovado o Projeto de Lei nº 3.802, de 2024. Sim: 340; Não: 117; Abstenção: 1; Total: 458.",Maioria,NaN,Sim
2,2460010-43,https://dadosabertos.camara.leg.br/api/v2/votacoes/2460010-43,PLEN,"Aprovado o Projeto de Lei nº 3.802, de 2024. Sim: 340; Não: 117; Abstenção: 1; Total: 458.",Governo,NaN,Sim
3,2460010-43,https://dadosabertos.camara.leg.br/api/v2/votacoes/2460010-43,PLEN,"Aprovado o Projeto de Lei nº 3.802, de 2024. Sim: 340; Não: 117; Abstenção: 1; Total: 458.",PSB,https://dadosabertos.camara.leg.br/api/v2/partidos/36832,Sim
4,2460010-43,https://dadosabertos.camara.leg.br/api/v2/votacoes/2460010-43,PLEN,"Aprovado o Projeto de Lei nº 3.802, de 2024. Sim: 340; Não: 117; Abstenção: 1; Total: 458.",PL,https://dadosabertos.camara.leg.br/api/v2/partidos/37906,Liberado
5,2460010-43,https://dadosabertos.camara.leg.br/api/v2/votacoes/2460010-43,PLEN,"Aprovado o Projeto de Lei nº 3.802, de 2024. Sim: 340; Não: 117; Abstenção: 1; Total: 458.",Bl UniPpFdrPsdbCid...,NaN,Sim
6,2460010-43,https://dadosabertos.camara.leg.br/api/v2/votacoes/2460010-43,PLEN,"Aprovado o Projeto de Lei nº 3.802, de 2024. Sim: 340; Não: 117; Abstenção: 1; Total: 458.",Oposição,NaN,Não
7,2460010-43,https://dadosabertos.camara.leg.br/api/v2/votacoes/2460010-43,PLEN,"Aprovado o Projeto de Lei nº 3.802, de 2024. Sim: 340; Não: 117; Abstenção: 1; Total: 458.",Novo,https://dadosabertos.camara.leg.br/api/v2/partidos/37901,Não
8,2460010-43,https://dadosabertos.camara.leg.br/api/v2/votacoes/2460010-43,PLEN,"Aprovado o Projeto de Lei nº 3.802, de 2024. Sim: 340; Não: 117; Abstenção: 1; Total: 458.",Bl MdbPsdRepPode,NaN,Sim
9,2460010-43,https://dadosabertos.camara.leg.br/api/v2/votacoes/2460010-43,PLEN,"Aprovado o Projeto de Lei nº 3.802, de 2024. Sim: 340; Não: 117; Abstenção: 1; Total: 458.",Fdr PSOL-REDE,NaN,Sim


In [46]:
# Orientações da votação de exemplo
df_orientacoes['idVotacao'] = df_orientacoes['idVotacao'].astype(str)
orientacoes_exemplo = df_orientacoes[df_orientacoes['idVotacao'] == str(id_votacao_exemplo)]

print(f"\n📋 ORIENTAÇÕES PARA VOTAÇÃO {id_votacao_exemplo}:")
if len(orientacoes_exemplo) > 0:
    # Usar nome correto da coluna (siglaBancada no arquivo votacoesOrientacoes)
    col_bancada = 'siglaBancada' if 'siglaBancada' in orientacoes_exemplo.columns else 'siglaPartidoBloco'
    print(orientacoes_exemplo[[col_bancada, 'orientacao']].to_string(index=False))
else:
    print("Nenhuma orientação encontrada para esta votação.")


📋 ORIENTAÇÕES PARA VOTAÇÃO 2367548-7:
         siglaBancada orientacao
     Bl MdbPsdRepPode        Sim
              Maioria        Sim
                 Novo        Sim
             Oposição        NaN
              Minoria        Sim
Bl UniPpFdrPsdbCid...        Sim
              Governo        Sim
      Fdr PT-PCdoB-PV        Sim
                   PL        Sim
                  PSB        NaN
        Fdr PSOL-REDE        Sim


## 8. ✅ Conclusão: Checklist de Viabilidade

In [47]:
print("="*80)
print("📋 CHECKLIST DE VIABILIDADE TÉCNICA")
print("="*80)

checks = [
    ("Proposições (PL, ementa, status)", len(df_proposicoes) > 0),
    ("Votações (data, resultado, placar)", len(df_votacoes) > 0),
    ("Votos individuais (deputado, partido, UF, SIM/NÃO)", len(df_votos) > 0),
    ("Link votação → proposição", len(df_votacoes_proposicoes) > 0),
    ("Orientação de bancada", len(df_orientacoes) > 0),
    ("Watch Item montável (JOIN funcionou)", len(df_watch) > 0),
]

todos_ok = True
for item, ok in checks:
    status = "✅" if ok else "❌"
    print(f"  {status} {item}")
    if not ok:
        todos_ok = False

print("\n" + "="*80)
if todos_ok:
    print("🎉 VEREDICTO: PROJETO É 100% VIÁVEL!")
    print("   Todos os dados necessários estão disponíveis na API da Câmara.")
else:
    print("⚠️  Alguns dados não foram encontrados. Investigar mais.")
print("="*80)

📋 CHECKLIST DE VIABILIDADE TÉCNICA
  ✅ Proposições (PL, ementa, status)
  ✅ Votações (data, resultado, placar)
  ✅ Votos individuais (deputado, partido, UF, SIM/NÃO)
  ✅ Link votação → proposição
  ✅ Orientação de bancada
  ✅ Watch Item montável (JOIN funcionou)

🎉 VEREDICTO: PROJETO É 100% VIÁVEL!
   Todos os dados necessários estão disponíveis na API da Câmara.


## 9. Próximos Passos Sugeridos

Agora que sabemos que os dados existem, os próximos passos seriam:

1. **Pipeline Bronze → Silver → Gold**
   - Bronze: Download diário dos CSVs
   - Silver: Normalização e JOIN das tabelas
   - Gold: Geração dos Watch Items com LLM

2. **Classificação de Stages**
   - Criar lógica para identificar texto-base vs. redação final vs. DTQ
   - Usar campo `descricao` da votação

3. **Integração com LLM**
   - Gerar "headline" a partir da ementa
   - Gerar "why it matters" explicando impacto

4. **Frontend MVP**
   - Feed de Watch Items
   - Página de detalhes da votação
   - Lista de votos por deputado

In [48]:
# Salvar dataset processado para uso futuro
output_path = os.path.join(DATA_DIR, 'watch_items_2024.csv')
df_watch.to_csv(output_path, index=False, encoding='utf-8')
print(f"💾 Dataset salvo em: {output_path}")

💾 Dataset salvo em: ./dados_camara/watch_items_2024.csv
